In [21]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from functions import net, MyDataset, train

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Prep the dataset

In [2]:
eb = pd.read_csv("../../data/clean/emobank_affectivetext_lex.csv")

# Ekman classes
classes = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]
features = ["VADER_neg", "VADER_neu", "VADER_pos", "VADER_compound"\
    , "TextBlob_polarity", "TextBlob_subjectivity", "AFINN"]
print("Number of features: ", len(features))
print("Number of classes: ", len(classes))

# Split into train, test, dev
eb_train = eb[eb["split"] == "train"]
eb_test = eb[eb["split"] == "test"]
eb_dev = eb[eb["split"] == "dev"]

# Create datasets
train_dataset = MyDataset(eb_train[features+classes])
test_dataset = MyDataset(eb_test[features+classes])
dev_dataset = MyDataset(eb_dev[features+classes])

Number of features:  7
Number of classes:  6


In [22]:
model = net(len(features), 10, len(classes), num_hidden=1)
train(model, train_dataset, dev_dataset, epochs=10)

c:\Users\marti\Documents\GitHub\EmotionAnalysis\notebooks\Methods 1 and 2\functions.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)